Copyright (c) 2021, salesforce.com, inc.\
All rights reserved.\
SPDX-License-Identifier: BSD-3-Clause\
For full license text, see the LICENSE file in the repo root or https://opensource.org/licenses/BSD-3-Clause

Try this notebook on [Colab](http://colab.research.google.com/github/salesforce/warp-drive/blob/master/tutorials/tutorial-5-training_with_warp_drive.ipynb)!

# ⚠️ PLEASE NOTE:
This notebook runs on a GPU runtime.\
If running on Colab, choose Runtime > Change runtime type from the menu, then select `GPU` in the 'Hardware accelerator' dropdown menu.

# Introduction

In this tutorial, we describe how to
- Use the WarpDrive framework to perform end-to-end training of multi-agent reinforcement learning (RL) agents.
- Visualize the behavior using the trained policies.

In case you haven't familiarized yourself with WarpDrive, please see the other tutorials we have prepared for you
- [WarpDrive basics](https://www.github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-1-warp_drive_basics.ipynb)
- [WarpDrive sampler](https://www.github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-2-warp_drive_sampler.ipynb)
- [WarpDrive reset and log controller](https://www.github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-3-warp_drive_reset_and_log.ipynb)

Please also see our [tutorial](https://www.github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-4-create_custom_environments.md) on creating your own RL environment in CUDA C. Once you have your own environment in CUDA C, this tutorial explains how to integrate it with the WarpDrive framework to perform training.

# Dependencies

You can install the warp_drive package using

- the pip package manager, OR
- by cloning the warp_drive package and installing the requirements.

We will install the latest version of WarpDrive using the pip package manager.

In [1]:
! pip install --quiet "rl-warp-drive>=1.6.5" "torch==1.10.*" "torchvision==0.11.*" "torchtext==0.11.*" "ffmpeg-python" 

In [2]:
import torch

assert torch.cuda.device_count() > 0, "This notebook needs a GPU to run!"

In [3]:
from warp_drive.env_wrapper import EnvWrapper
from warp_drive.training.trainer import Trainer
from warp_drive.utils.common import get_project_root

from example_envs.tag_continuous.tag_continuous import TagContinuous
from example_envs.tag_continuous.generate_rollout_animation import (
    generate_tag_env_rollout_animation,
)

In [4]:
from gym.spaces import Discrete, MultiDiscrete
from IPython.display import HTML
import yaml
import numpy as np

In [5]:
# Set logger level e.g., DEBUG, INFO, WARNING, ERROR
import logging

logging.getLogger().setLevel(logging.ERROR)

# Training the continuous version of Tag with WarpDrive

We will now explain how to train your environments using WarpDrive in just a few steps. For the sake of exposition, we consider the continuous version of Tag.

For your reference, there is also an example end-to-end RL training script [here](https://github.com/salesforce/warp-drive/blob/master/warp_drive/training/example_training_script.py) that contains all the steps below. It can be used to set up your own custom training pipeline. Invoke training by using
```shell
python warp_drive/training/example_training_script.py --env <ENV-NAME>
```
where `<ENV-NAME>` can be `tag_gridworld` or `tag_continuous` (or any new env that you build).

## Step 1: Specify a set of run configurations for your experiments.

In order to run the training for these environments, we first need to specify a *run config*, which comprises the set of environment, training, and model parameters.

Note: there are also some default configurations in 'warp_drive/training/run_configs/default_configs.yaml', and the run configurations you provide will override them.

For this tutorial, we will use the configuration [here](assets/tag_continuous_training/run_config.yaml). Specifically, we'll use $5$ taggers and $100$ runners in a $20 \times 20$ square grid. The taggers and runners have the same skill level, i.e., the runners can move just as fast as the taggers.

The sequence of snapshots below shows a sample realization of the game with randomly chosen agent actions. The $5$ taggers are marked in pink, while the $100$ blue agents are the runners. The snapshots are taken at 1) the beginning of the episode ($t=0$), 2) timestep $250$, and 3) end of the episode ($t=500$). Only $20\%$ of the runners remain at the end of the episode.

<img src="assets/tag_continuous_training/t=0.png" width="250" height="250"/> <img src="assets/tag_continuous_training/t=250.png" width="250" height="250"/> <img src="assets/tag_continuous_training/t=500.png" width="250" height="250"/>

We train the agents using $200$ environments or simulations running in parallel. With WarpDrive, each simulation runs on sepate GPU blocks.

There are two separate policy networks used for the tagger and runner agents. Each network is a fully-connected model with two layers each of $256$ dimensions. We use the Advantage Actor Critic (A2C) algorithm for training. WarpDrive also currently provides the option to use the Proximal Policy Optimization (PPO) algorithm instead.

In [6]:
# Load the run config.

# Here we show an example configures

CFG = """
# Sample YAML configuration for the tag continuous environment
name: "tag_continuous"

# Environment settings
env:
    num_taggers: 5
    num_runners: 100
    grid_length: 20
    episode_length: 500
    max_acceleration: 0.1
    min_acceleration: -0.1
    max_turn: 2.35  # 3*pi/4 radians
    min_turn: -2.35  # -3*pi/4 radians
    num_acceleration_levels: 10
    num_turn_levels: 10
    skill_level_runner: 1
    skill_level_tagger: 1
    seed: 274880
    use_full_observation: False
    runner_exits_game_after_tagged: True
    num_other_agents_observed: 10
    tag_reward_for_tagger: 10.0
    tag_penalty_for_runner: -10.0
    step_penalty_for_tagger: -0.00
    step_reward_for_runner: 0.00
    edge_hit_penalty: -0.0
    end_of_game_reward_for_runner: 1.0
    tagging_distance: 0.02

# Trainer settings
trainer:
    num_envs: 400 # number of environment replicas
    train_batch_size: 10000 # total batch size used for training per iteration (across all the environments)
    num_episodes: 500 # number of episodes to run the training for (can be arbitrarily high)
# Policy network settings
policy: # list all the policies below
    runner:
        to_train: True # flag indicating whether the model needs to be trained
        algorithm: "A2C" # algorithm used to train the policy
        gamma: 0.98 # discount rate gamms
        lr: 0.005 # learning rate
        vf_loss_coeff: 1 # loss coefficient for the value function loss
        entropy_coeff:
        - [0, 0.5]
        - [2000000, 0.05]
        model: # policy model settings
            type: "fully_connected" # model type
            fc_dims: [256, 256] # dimension(s) of the fully connected layers as a list
            model_ckpt_filepath: "" # filepath (used to restore a previously saved model)
    tagger:
        to_train: True
        algorithm: "A2C"
        gamma: 0.98
        lr: 0.002
        vf_loss_coeff: 1
        model:
            type: "fully_connected"
            fc_dims: [256, 256]
            model_ckpt_filepath: ""

# Checkpoint saving setting
saving:
    metrics_log_freq: 100 # how often (in iterations) to print the metrics
    model_params_save_freq: 5000 # how often (in iterations) to save the model parameters
    basedir: "/tmp" # base folder used for saving
    name: "tag_continuous"
    tag: "100runners_5taggers"

"""

run_config = yaml.safe_load(CFG)

## Step 2: Create the environment object using WarpDrive's envWrapper.

### Important! Ensure that 'use_cuda' is set to True below (in order to run the simulation on the GPU).

In [7]:
env_wrapper = EnvWrapper(
    env_obj=TagContinuous(**run_config["env"]),
    num_envs=run_config["trainer"]["num_envs"],
    use_cuda=True,
)

/home/jupyter/.local/lib/python3.8/site-packages/gym/utils/seeding.py:41: DeprecationWarning: WARN: Function `rng.rand(*size)` is marked as deprecated and will be removed in the future. Please use `Generator.random(size)` instead.
  deprecation(


Creating the env wrapper initializes the CUDA data manager and pushes some reserved data arrays to the GPU. It also initializes the CUDA function manager, and loads some WarpDrive library CUDA kernels.

## Step 3: Specify a mapping from the policy to agent indices.

Next, we will need to map each trainable policy to the agent indices that are using it. As such, we have the tagger and runner policies, and we will map those to the corresponding agents.

In [8]:
policy_tag_to_agent_id_map = {
    "tagger": list(env_wrapper.env.taggers),
    "runner": list(env_wrapper.env.runners),
}

Note that if you wish to use just a single policy across all the agents (or if you wish to use many other policies), you will need to update the run configuration as well as the policy_to_agent_id_mapping.

For example, for using a shared policy across all agents (say `shared_policy`), for example, you can just use the run configuration as
```python
    "policy": {
        "shared_policy": {
            "to_train": True,
            ...
        },
    },
```
and also set all the agent ids to use this shared policy
```python
    policy_tag_to_agent_id_map = {
        "shared_policy": np.arange(envObj.env.num_agents),
    }
```

**Importantly, make sure the `policy` keys and the `policy_tag_to_agent_id_map` keys are identical.**

## Step 4: Create the Trainer object.

In [9]:
trainer = Trainer(env_wrapper, run_config, policy_tag_to_agent_id_map)

The `Trainer` class also takes in a few optional arguments that will need to be correctly set, if required.
- `create_separate_placeholders_for_each_policy`: a flag indicating whether there exist separate observations, actions and rewards placeholders, for each policy, used in the step() function and during training. When there's only a single policy, this flag will be False. It can also be True when there are multiple policies, yet all the agents have the same obs and action space shapes, so we can share the same placeholder. Defaults to "False".
- `obs_dim_corresponding_to_num_agents`: indicative of which dimension in the observation corresponds to the number of agents, as designed in the step function. It may be "first" or "last". In other words, observations may be shaped (num_agents, feature_dims) or (feature_dims, num_agents). This is required in order for WarpDrive to process the observations correctly. This is only relevant when a single obs key corresponds to multiple agents. Defaults to "first".
- `num_devices`: number of GPU devices used for (distributed) training. Defaults to 1.
- `device_id`: the device ID. This is set in the context of multi-GPU training.
- `results_dir`: (optional) name of the directory to save results into. If this is not set, the current timestamp will be used instead.
- `verbose`: if False, training metrics are not printed to the screen. Defaults to True.

When the trainer object is created, the environment(s) are reset and all the relevant data arrays (e.g., "loc_x", "loc_y, "speed") are automatically pushed from the CPU to the GPU (just once). Additionally, the observation, reward, action and done flag data array sizes are determined and the array placeholders are also automatically pushed to the GPU. After training begins, these arrays are only updated in-place, and there's no data transferred back to the CPU.

# Visualizing the trainer policies

## Visualizing an episode roll-out before training 

We have created a helper function (`generate_tag_env_rollout_animation`) in order to visualize an episode rollout. Internally, this function uses the WarpDrive module's `fetch_episode_states` to fetch the data arrays on the GPU for the duration of an entire episode. Specifically, we fetch the state arrays pertaining to agents' x and y locations on the plane and indicators on which agents are still active in the game, and will use these to visualize an episode roll-out. Note that this function may be invoked at any time during training, and it will use the state of the policy models at that time to sample actions and generate the visualization.

In [12]:
!pip install ffmpeg-python

Defaulting to user installation because normal site-packages is not writeable


In [10]:
# Visualize the entire episode roll-out
anim = generate_tag_env_rollout_animation(trainer)
HTML(anim.to_html5_video())

RuntimeError: Requested MovieWriter (ffmpeg) not available

In the visualization above, the large purple dots represent the taggers, while the smaller blue dots represent the runners. Before training, the runners and taggers move around randomly, and that results in a lot of the runners getting tagged, just by chance.

## Step 5: Perform training

Training is performed by calling trainer.train(). We run training for just $500$ episodes, as specified in the run configuration.

In [11]:
trainer.train()



Device: 0
Iterations Completed                    : 1 / 25
Speed performance stats
Mean policy eval time per iter (ms)     :      87.12
Mean action sample time per iter (ms)   :       5.64
Mean env. step time per iter (ms)       :     583.55
Mean training time per iter (ms)        :     922.47
Mean total time per iter (ms)           :    1605.53
Mean steps per sec (policy eval)        :  114778.01
Mean steps per sec (action sample)      : 1772345.74
Mean steps per sec (env. step)          :   17136.58
Mean steps per sec (training time)      :   10840.45
Mean steps per sec (total)              :    6228.48
Metrics for policy 'runner'
VF loss coefficient                     :    1.00000
Entropy coefficient                     :    0.50000
Total loss                              :   -0.86481
Policy loss                             :   -1.75696
Value function loss                     :    3.28871
Mean rewards                            :   -0.03558
Max. rewards                           

As training happens, we log the speed performance numbers and the metrics for all the trained policies every `metrics_log_freq` iterations. The training results and the model checkpoints are also saved on a timely (as specified in the run configuration parameters `model_params_save_freq`) basis.

## Visualize an episode-rollout after training (for about 2M episodes)

We can also initialize the trainer model parameters using saved model checkpoints via the `load_model_checkpoint` API. With this, we will be able to fetch the episode states for a trained model, for example. We will now visualize an episode roll-out using trained tagger and runner policy model weights (trained for $2$M episodes or $1$B steps), that are located in [this](assets/tag_continuous_training/) folder.

In [13]:
!pwd

/home/jupyter/warp-drive/tutorials


In [ ]:
#make sure to fix the path to the tutorial in the next cell based on this example
"""
trainer.load_model_checkpoint(
    {
        "tagger": "FIX_ME/tutorials/assets/tag_continuous_training/tagger_1000010000.state_dict",
        "runner": "FIXME/tutorials/assets/tag_continuous_training/runner_1000010000.state_dict",
    }
"""

In [14]:
trainer.load_model_checkpoint(
    {
        "tagger": "/home/jupyter/warp-drive/tutorials/assets/tag_continuous_training/tagger_1000010000.state_dict",
        "runner": "/home/jupyter/warp-drive/tutorials/assets/tag_continuous_training/runner_1000010000.state_dict",
    }
)

[Device 0]: Loading the provided trainer model checkpoints. 
[Device 0]: Loading the 'tagger' torch model from the previously saved checkpoint: '/home/jupyter/warp-drive/tutorials/assets/tag_continuous_training/tagger_1000010000.state_dict' 
[Device 0]: Updating the timestep for the 'tagger' model to 1000010000. 
[Device 0]: Loading the 'runner' torch model from the previously saved checkpoint: '/home/jupyter/warp-drive/tutorials/assets/tag_continuous_training/runner_1000010000.state_dict' 
[Device 0]: Updating the timestep for the 'runner' model to 1000010000. 


In [15]:
# Visualize the entire episode roll-out
anim = generate_tag_env_rollout_animation(trainer)
HTML(anim.to_html5_video())

RuntimeError: Requested MovieWriter (ffmpeg) not available

After training, the runners learn to run away from the taggers, and the taggers learn to chase them; there are some instances where we see that taggers also team up to chase and tag the runners.

You have now seen how to train an entire multi-agent RL pipeline end-to-end. Please see the next [tutorial](https://www.github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-6-scaling_up_training_with_warp_drive.md) on scaling up training.

We also have a [trainer](https://github.com/salesforce/warp-drive/blob/master/warp_drive/training/lightning_trainer.py) compatible with [Pytorch Lightning](https://www.pytorchlightning.ai/) and have prepared a tutorial on training with WarpDrive and Pytorch Lightning [here](https://github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-7-training_with_warp_drive_and_pytorch_lightning.ipynb).

## Step 6: Gracefully shut down the trainer object

In [16]:
# Close the trainer to clear up the CUDA memory heap
trainer.graceful_close()

[Device 0]: Trainer exits gracefully 


# Learn More and Explore our Tutorials!

For your reference, all our tutorials are here:
1. [WarpDrive basics](https://www.github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-1-warp_drive_basics.ipynb)
2. [WarpDrive sampler](https://www.github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-2-warp_drive_sampler.ipynb)
3. [WarpDrive reset and log](https://www.github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-3-warp_drive_reset_and_log.ipynb)
4. [Creating custom environments](https://www.github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-4-create_custom_environments.md)
5. [Training with WarpDrive](https://www.github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-5-training_with_warp_drive.ipynb)
6. [Scaling Up training with WarpDrive](https://www.github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-6-scaling_up_training_with_warp_drive.md)
7. [Training with WarpDrive + Pytorch Lightning](https://github.com/salesforce/warp-drive/blob/master/tutorials/tutorial-7-training_with_warp_drive_and_pytorch_lightning.ipynb)